![Group Chat with Agents](Image/i5.png)

In [1]:
from dotenv import load_dotenv
load_dotenv()
# get openai-key
import os
from autogen import ConversableAgent
openai_key = os.getenv('OPENAI_API_KEY')
llm_config={"model":"gpt-4-turbo"}

In [2]:
llm_config={"model": "gpt-4-turbo"}

task = "Write a blogpost about the stock price performance of "\
"Nvidia in the past month. Today's date is 2024-04-23."

- User_proxy veya Admin: Kullanıcının rapor hakkında yorum yapmasına ve yazarın raporda düzeltme yapmasını istemesine olanak tanır.
- planner: Görevi tamamlamak için gereken ilgili bilgileri belirler.
- engineer: Planlayıcının tanımladığı planı kullanarak kod yazar.
- Yürütücü (Executor): Mühendis tarafından yazılan kodu yürütür.
- Yazar (Writer): Raporu yazar.

In [4]:
import autogen
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code based on the plan "
    "provided by the planner.",
)
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="Writer."
    "Write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog."
)

define Group Conversation

In [5]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

start group conversation

In [6]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about Nvidia's stock price performance over the past month, you will need the following information:

1. **Historical Stock Prices**: Access daily closing prices for Nvidia from March 23, 2024, to April 23, 2024. This data should include at least the open, high, low, and close prices for each trading day.

2. **Market News**: Gather any relevant news articles or reports about Nvidia or the broader tech sector that could have influenced stock prices during this time period.

3. **Financial Analysis**: Obtain any recent financial reports or earnings releases from Nvidia if they were released within this last month, as these documents can significantly affect stock prices.

4